In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from pathlib import Path
import pyarrow.dataset as ds
from pgpq import ArrowToPostgresBinaryEncoder
import psycopg
from tqdm import tqdm
import duckdb

from splink.duckdb.linker import DuckDBLinker

from src.data import utils as du
import src.locations as loc
from src.config import settings

CLUSTER_PATH = Path(loc.DATA_SUBDIR['processed']) / 'company-matching__full' / 'clusters.parquet' 

# Using Splink with physical duckdb

Gonna try and run it off the file system. Raw db about 1GB pre-Splink.

In [3]:
con = du.get_duckdb_connection()

In [4]:
table_name = []
table_alias = []

for i in con.query("select * from table_alias_lookup;").fetchall():
    table_alias.append(i[0])
    table_name.append(i[1])

In [5]:
linker = DuckDBLinker(
    table_name,
    settings_dict=settings,
    connection=con,
    input_table_aliases=table_alias,
)

## Train

In [6]:
linker.estimate_probability_two_random_records_match(
    "l.name_unusual_tokens = r.name_unusual_tokens",
    recall=0.7,
)

Probability two random records match is estimated to be  3.24e-06.
This means that amongst all possible pairwise record comparisons, one in 309,025.51 are expected to match.  With 40,009,433,095,801 total possible comparisons, we expect a total of around 129,469,675.71 matching pairs


In [7]:
linker.estimate_u_using_random_sampling(max_pairs=1e7)

----- Estimating u probabilities using random sampling -----
u probability not trained for comp_num_clean - Exact match (comparison vector value: 2). This usually means the comparison level was never observed in the training data.

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - comp_num_clean (some u values are not trained, no m values are trained).
    - name_unusual_tokens (no m values are trained).
    - postcode (no m values are trained).


In [8]:
linker.estimate_m_from_label_column("comp_num_clean")
m_by_name_and_postcode_area = """
    l.name_unusual_tokens = r.name_unusual_tokens
    and l.postcode_area = r.postcode_area
"""
linker.estimate_parameters_using_expectation_maximisation(
    m_by_name_and_postcode_area
)

---- Estimating m probabilities using from column comp_num_clean -----
m probability not trained for comp_num_clean - Jaro_winkler_similarity >= 0.75 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
m probability not trained for comp_num_clean - All other comparisons (comparison vector value: 0). This usually means the comparison level was never observed in the training data.

Your model is not yet fully trained. Missing estimates for:
    - comp_num_clean (some u values are not trained, some m values are not trained).

----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:

    l.name_unusual_tokens = r.name_unusual_tokens
    and l.postcode_area = r.postcode_area


Parameter estimates will be made for the following comparison(s):
    - comp_num_clean
    - postcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
 

<EMTrainingSession, blocking on 
    l.name_unusual_tokens = r.name_unusual_tokens
    and l.postcode_area = r.postcode_area
, deactivating comparisons name_unusual_tokens>

## Predict

In [9]:
predictions = linker.predict(threshold_match_probability=0.7)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'comp_num_clean':
    u values not fully trained


## Cluster

In [10]:
predict_table = con.query("""
    select table_name
    from information_schema.tables
    where table_name like '%predict%';
""").fetchone()[0]
predictions = linker.register_table(predict_table, predict_table)

In [ ]:
clusters = linker.cluster_pairwise_predictions_at_threshold(
    predictions,
    threshold_match_probability=0.7,
    pairwise_formatting=True,
    filter_pairwise_format_for_clusters=False,
)

Completed iteration 1, root rows count 11871
Completed iteration 2, root rows count 208
Completed iteration 3, root rows count 97
Completed iteration 4, root rows count 3
Completed iteration 5, root rows count 0


In [ ]:
clusters.physical_name

## Review

In [ ]:
con.query(f"select count(*) from {predict_table};")

In [ ]:
con.query(f"select count(*) from {clusters.physical_name};")

In [ ]:
con.query("pragma database_size;")

In [ ]:
con.query("""
    select table_name
    from information_schema.tables;
""")

## Export

In [ ]:
con.query(f"""
    copy (
        select
            src_tbl.table_name as source,
            src_id.unique_id as source_id,
            cl.source_cluster,
            tgt_tbl.table_name as target,
            tgt_id.unique_id as target_id,
            cl.target_cluster,
            cl.match_probability
        from (
            select
                source_dataset_l as source,
                unique_id_l as source_id,
                cluster_id_l as source_cluster,
                source_dataset_r as target,
                unique_id_r as target_id,
                cluster_id_r as target_cluster,
                match_probability
            from
                { clusters.physical_name }
            union
            select
                source_dataset_r as source,
                unique_id_r as source_id,
                cluster_id_r as source_cluster,
                source_dataset_l as target,
                unique_id_l as target_id,
                cluster_id_l as target_cluster,
                match_probability
            from
                { clusters.physical_name }
        ) cl
        join table_alias_lookup src_tbl on
            (cl.source = src_tbl.id)
        join unique_id_lookup src_id on
            (cl.source_id = src_id.id)
        join table_alias_lookup tgt_tbl on
            (cl.target = tgt_tbl.id)
        join unique_id_lookup tgt_id on
            (cl.target_id = tgt_id.id)
    )
    to '{CLUSTER_PATH.as_posix()}'
    (format parquet);
""")

In [29]:
%%time

to_write = ds.dataset(CLUSTER_PATH)
encoder = ArrowToPostgresBinaryEncoder(to_write.schema)
pg_schema = encoder.schema()
cols = [f'"{col_name}" {col.data_type.ddl()}' for col_name, col in pg_schema.columns]
ddl = f"create temp table data ({','.join(cols)})"

with psycopg.connect("postgres://") as conn:
    with conn.cursor() as cur:
        cur.execute(ddl) 
        with cur.copy("copy data from stdin with (format binary)") as copy:
            copy.write(encoder.write_header())
            for batch in tqdm(to_write.to_batches()):
                copy.write(encoder.write_batch(batch))
            copy.write(encoder.finish())
        cur.execute("drop table if exists \"_user_eaf4fd9a\".\"lookup\"")
        cur.execute("""
            create table \"_user_eaf4fd9a\".\"lookup\" as 
            select * from data
        """)

1294it [04:10,  5.17it/s]


CPU times: user 1min 8s, sys: 11.1 s, total: 1min 19s
Wall time: 15min 42s


In [30]:
%%time

with psycopg.connect("postgres://") as conn:
    with conn.cursor() as cur:
        cur.execute("drop index if exists \"idx_wl_lookup_src_tgt\"")
        cur.execute("drop index if exists \"idx_wl_lookup_src_tgt_id\"")
        
        cur.execute("create index \"idx_wl_lookup_src_tgt\" on \"_user_eaf4fd9a\".\"lookup\"(source, target)")
        cur.execute("create index \"idx_wl_lookup_src_tgt_id\" on \"_user_eaf4fd9a\".\"lookup\"(source_id, target_id)")

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=datasets-1-aurora-2.cq9e4gwvtop7.eu-west-2.rds.amazonaws.com user=user_will_langdale_trade_gov_uk_crzjk database=public_datasets_1) at 0x7f65c14ab9e0>

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=datasets-1-aurora-2.cq9e4gwvtop7.eu-west-2.rds.amazonaws.com user=user_will_langdale_trade_gov_uk_crzjk database=public_datasets_1) at 0x7f65c14ab9e0>

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=datasets-1-aurora-2.cq9e4gwvtop7.eu-west-2.rds.amazonaws.com user=user_will_langdale_trade_gov_uk_crzjk database=public_datasets_1) at 0x7f65c14ab9e0>

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=datasets-1-aurora-2.cq9e4gwvtop7.eu-west-2.rds.amazonaws.com user=user_will_langdale_trade_gov_uk_crzjk database=public_datasets_1) at 0x7f65c14ab9e0>

## Debug

In [ ]:
duckdb.sql(f"select * from '{CLUSTER_PATH.as_posix()}' limit 5;")